In [1]:
import pandas as pd
import numpy as np
import requests
import json
# from confluent_kafka import Producer
from kafka import KafkaProducer, KafkaConsumer
from confluent_kafka import Producer, Consumer, KafkaError
import time
from time import sleep
from datetime import datetime
import logging
import pyarrow as pa 
import pyarrow.parquet as pq
import re
import boto3
import dotenv
from dotenv import load_dotenv
import os

In [176]:
load_dotenv()

True

In [177]:
# access_key = os.getenv("ACCESS_KEY")
# secret_key = os.getenv("SECRET_ACCESS_KEY")

In [178]:
aws_management_console = boto3.session.Session(profile_name='default')
iam_console_resource = aws_management_console.resource('iam') # resource obj
iam_console_client = aws_management_console.client('iam') # client obj

for each_user in iam_console_resource.users.all():
    print(each_user.name)

DEBUG:botocore.hooks:Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane
DEBUG:botocore.hooks:Changing event name from before-call.apigateway to before-call.api-gateway
DEBUG:botocore.hooks:Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict
DEBUG:botocore.hooks:Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration
DEBUG:botocore.hooks:Changing event name from before-parameter-build.route53 to before-parameter-build.route-53
DEBUG:botocore.hooks:Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search
DEBUG:botocore.hooks:Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-section to docs.*.auto-scaling.CreateLaunchConfiguration.complete-section
DEBUG:botocore.hooks:Changing event name from before-parameter-buil

boto3
experimentIAM


In [179]:
print(aws_management_console.get_available_resources())

['cloudformation', 'cloudwatch', 'dynamodb', 'ec2', 'glacier', 'iam', 'opsworks', 's3', 'sns', 'sqs']


In [180]:
for each in iam_console_client.list_users()['Users']:
    print(each['UserName'])

DEBUG:botocore.hooks:Event before-parameter-build.iam.ListUsers: calling handler <function generate_idempotent_uuid at 0x000001D15956CEA0>
DEBUG:botocore.regions:Calling endpoint provider with parameters: {'Region': 'us-east-1', 'UseDualStack': False, 'UseFIPS': False}
DEBUG:botocore.regions:Endpoint provider result: https://iam.amazonaws.com
DEBUG:botocore.regions:Selecting from endpoint provider's list of auth schemes: "sigv4". User selected auth scheme is: "None"
DEBUG:botocore.regions:Selected auth type "v4" as "v4" with signing context params: {'region': 'us-east-1', 'signing_name': 'iam'}
DEBUG:botocore.hooks:Event before-call.iam.ListUsers: calling handler <function add_recursion_detection_header at 0x000001D159542660>
DEBUG:botocore.hooks:Event before-call.iam.ListUsers: calling handler <function inject_api_version_header_if_needed at 0x000001D15956E980>
DEBUG:botocore.endpoint:Making request for OperationModel(name=ListUsers) with params: {'url_path': '/', 'query_string': '', 

boto3
experimentIAM


In [181]:
def extract_words_from_urls(urls):
    words = []
    pattern = r"en/(\w+)"
    for url_info in urls:
        url = url_info['url']
        match = re.search(pattern, url)
        if match:
            word = match.group(1)
            words.append(word)
    return words

def fetch_json_data(url):
    response = requests.get(url)
    return response.json()

def import_json_to_parquet(urls, parquet_file):
    # Fetch JSON data from each URL and store in a list
    json_data_list = []
    for url_info in urls:
        url = url_info['url']
        json_data = fetch_json_data(url)
        json_data_list.append(json_data)

    # Create DataFrames from JSON data and concatenate them
    dfs = [pd.DataFrame(data) for data in json_data_list]
    combined_df = pd.concat(dfs, ignore_index=True)

    # Write combined DataFrame to Parquet file
    table = pa.Table.from_pandas(combined_df)
    pq.write_table(table, parquet_file)

def create_variables_from_words(word):
    globals()[word] = word
    return globals()[word]

def produce_parquet_to_kafka(producer, topic, parquet_file):
    table = pq.read_table(parquet_file)
    for batch in table.to_batches():
        binary_data = batch.to_pandas().to_json().encode('utf-8')  # Convert to binary format
        producer.produce(topic, value=binary_data)

    producer.flush()  # Wait for all messages to be delivered

def upload_binary_data_to_s3(binary_data, bucket_name, object_key):
    s3_client = boto3.client('s3')
    try:
        response = s3_client.put_object(Bucket=bucket_name, Key=object_key, Body=binary_data)
        print("Binary data uploaded successfully to S3")
    except Exception as e:
        print(f"Error uploading binary data to S3: {e}")

def consume_and_upload_to_s3(consumer, s3_bucket):
    try:
        while True:
            message = consumer.poll(1.0)  # Poll for messages with a timeout of 1 second
            if message is None:
                continue
            if message.error():
                if message.error().code() == KafkaError._PARTITION_EOF:
                    continue
                else:
                    print(f"Error while consuming message: {message.error()}")
                    break

            # Upload binary data to S3
            upload_binary_data_to_s3(message.value(), s3_bucket, f"parquet_file_{message.offset()}")

    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()

In [182]:
# logging.basicConfig(level=logging.DEBUG)

urls = [
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_information.json", "name": "system_information"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/station_information.json", "name": "station_information"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/station_status.json", "name": "station_status"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/free_bike_status.json", "name": "free_bike_status"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_hours.json", "name": "system_hours"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_calendar.json", "name": "system_calendar"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_regions.json", "name": "system_regions"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_pricing_plans.json", "name": "system_pricing_plans"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/system_alerts.json", "name": "system_alerts"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/gbfs_versions.json", "name": "gbfs_versions"},
    {"url": "https://gbfs.lyft.com/gbfs/2.3/dca-cabi/en/vehicle_types.json", "name": "vehicle_types"}
]

parquet_file = "../data/data.parquet"
words = extract_words_from_urls(urls)

#### System information

In [183]:
globals()[words[0]] = fetch_json_data(urls[0]['url'])
globals()[words[0]] = dict(globals()[words[0]]['data'])
system_information_df = pd.DataFrame(globals()[words[0]], index=[0])
system_information_df

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gbfs.lyft.com:443
DEBUG:urllib3.connectionpool:https://gbfs.lyft.com:443 "GET /gbfs/2.3/dca-cabi/en/system_information.json HTTP/1.1" 200 166


,system_id,timezone,name,language
0,lyft_dca-cabi,America/Los_Angeles,Lyft,en


In [184]:
system_information_df.to_parquet("../data/system_information.parquet")

In [185]:
if __name__ == "__main__":
    kafka_conf = {
        'bootstrap.servers': 'localhost:9092',
    }
    producer = Producer(kafka_conf)

    parquet_file = '../data/system_information.parquet'
    kafka_topic = 'system_information_parquet_data_topic'

    produce_parquet_to_kafka(producer, kafka_topic, parquet_file)

In [ ]:
if __name__ == "__main__":
    kafka_conf = {
        'bootstrap.servers': 'localhost:9092',
        'group.id': 'parquet_consumer_group',
        'auto.offset.reset': 'earliest'
    }
    consumer = Consumer(kafka_conf)
    consumer.subscribe(['system_information_parquet_data_topic'])

    s3_bucket = 'capitalbikesharegbfs'

    consume_and_upload_to_s3(consumer, s3_bucket)

#### Station information

In [ ]:
i=1
globals()[words[i]] = fetch_json_data(urls[i]['url'])
globals()[words[i]] = dict(globals()[words[i]]['data'])
globals()[words[i]] = globals()[words[i]]['stations']
station_information_df = pd.DataFrame()
for n in range(len(globals()[words[i]])):
    station_information_df = pd.concat([station_information_df, pd.DataFrame(globals()[words[i]][n])])
    
station_information_df.head()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gbfs.lyft.com:443
DEBUG:urllib3.connectionpool:https://gbfs.lyft.com:443 "GET /gbfs/2.3/dca-cabi/en/station_information.json HTTP/1.1" 200 233057


,short_name,lon,rental_uris,lat,station_id,capacity,name,region_id
ios,31958,-77.048043,https://dc.lft.to/lastmile_qr_scan,38.800703,472dee60-f045-4f2e-9807-193e3ae9b661,12,S Washington St & Wilkes St,40
android,31958,-77.048043,https://dc.lft.to/lastmile_qr_scan,38.800703,472dee60-f045-4f2e-9807-193e3ae9b661,12,S Washington St & Wilkes St,40
ios,31704,-76.941877,https://dc.lft.to/lastmile_qr_scan,38.901385,082480f1-1f3f-11e7-bf6b-3863bb334450,11,Nannie Helen Burroughs & Minnesota Ave NE,42
android,31704,-76.941877,https://dc.lft.to/lastmile_qr_scan,38.901385,082480f1-1f3f-11e7-bf6b-3863bb334450,11,Nannie Helen Burroughs & Minnesota Ave NE,42
ios,31625,-77.001828,https://dc.lft.to/lastmile_qr_scan,38.881185,08251448-1f3f-11e7-bf6b-3863bb334450,19,3rd & G St SE,42


In [ ]:
i=2
globals()[words[i]] = fetch_json_data(urls[i]['url'])
globals()[words[i]] = dict(globals()[words[i]]['data'])
globals()[words[i]] = globals()[words[i]]['stations']
station_status_df = pd.DataFrame()
for n in range(len(globals()[words[i]])):
    station_status_df = pd.concat([station_status_df, pd.DataFrame(globals()[words[i]][n])])
    
station_status_df.head()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gbfs.lyft.com:443
DEBUG:urllib3.connectionpool:https://gbfs.lyft.com:443 "GET /gbfs/2.3/dca-cabi/en/station_status.json HTTP/1.1" 200 335686


,vehicle_types_available,last_reported,num_ebikes_available,is_renting,num_bikes_disabled,station_id,num_docks_disabled,num_docks_available,num_bikes_available,is_returning,num_scooters_available,num_scooters_unavailable,is_installed
0,"{'count': 0, 'vehicle_type_id': '1'}",1713254099,2,1,0,472dee60-f045-4f2e-9807-193e3ae9b661,0,10,2,1,0.0,0.0,1
1,"{'count': 2, 'vehicle_type_id': '2'}",1713254099,2,1,0,472dee60-f045-4f2e-9807-193e3ae9b661,0,10,2,1,0.0,0.0,1
0,"{'count': 3, 'vehicle_type_id': '1'}",1713254097,0,1,0,082480f1-1f3f-11e7-bf6b-3863bb334450,1,7,3,1,0.0,0.0,1
1,"{'count': 0, 'vehicle_type_id': '2'}",1713254097,0,1,0,082480f1-1f3f-11e7-bf6b-3863bb334450,1,7,3,1,0.0,0.0,1
0,"{'count': 3, 'vehicle_type_id': '1'}",1713254101,0,1,2,08251448-1f3f-11e7-bf6b-3863bb334450,0,14,3,1,0.0,0.0,1


#### Free bike status

In [ ]:
i=3
globals()[words[i]] = fetch_json_data(urls[i]['url'])
globals()[words[i]] = dict(globals()[words[i]]['data'])
globals()[words[i]] = globals()[words[i]]['bikes']
free_bike_status_df = pd.DataFrame()
for n in range(len(globals()[words[i]])):
    free_bike_status_df = pd.concat([free_bike_status_df, pd.DataFrame(globals()[words[i]][n])])
    
free_bike_status_df.head()

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): gbfs.lyft.com:443
DEBUG:urllib3.connectionpool:https://gbfs.lyft.com:443 "GET /gbfs/2.3/dca-cabi/en/free_bike_status.json HTTP/1.1" 200 255349


,lon,rental_uris,lat,bike_id,vehicle_type_id,is_reserved,current_range_meters,is_disabled
ios,-76.963496,https://dc.lft.to/lastmile_qr_scan,38.879067,fda05c83a22e14b1502510a5a9c6bd29,2,0,23818.2912,0
android,-76.963496,https://dc.lft.to/lastmile_qr_scan,38.879067,fda05c83a22e14b1502510a5a9c6bd29,2,0,23818.2912,0
ios,-77.075975,https://dc.lft.to/lastmile_qr_scan,38.996167,3b4310bfa5cfd2e764c9194b695213b8,2,0,36049.3056,0
android,-77.075975,https://dc.lft.to/lastmile_qr_scan,38.996167,3b4310bfa5cfd2e764c9194b695213b8,2,0,36049.3056,0
ios,-77.019677,https://dc.lft.to/lastmile_qr_scan,38.954175,f94695b5f46f0ac6bf7ffc2f0535209d,2,0,22048.0128,0
